**Hidden Markov models for cracking codes**

In this exercise you have to make a partially built HMM work and use it to solve some simple substitution ciphers. Plaintext data is provided in 'plaintext' directory. Encrypted data is in 'encrypted'. Some of the texts were originally English some of them were Russian; the sequences are also of different lengths. 

This homework is worth **15 points** and is due by the next class (**24th Oct.**), please submit the results of the **TASK 5** (a list of files and names of the author/work) to Anytask in the following format: 'filename author' where 'filename' is a file from "encrypted/\*_encrypted.txt" and 'author' is a file from "plaintext/\*.txt" (not including 'english.txt', 'russian.txt' or 'all.txt') which best matches the decrypted text.




In [7]:
# Utilities for loading data from file and converting characters to integers and back.
import numpy as np
    
def get_char_to_int_mapping(path):
    # Load data from path and get mapping from characters to integers and back.
    characters = set()
    for line in open(path):
        characters.update(set([c for c in line.strip()]))
    char_to_int_mapping = dict([(char, i) for i, char in enumerate(sorted(list(characters)))])
    int_to_char_mapping = [char for char, i in char_to_int_mapping.items()]
    return char_to_int_mapping, int_to_char_mapping

def load_sequences(path, char_to_int_mapping):
    # Load data from path and map to integers using mapping.
    return [[char_to_int_mapping[c] for c in line.strip()] for line in open(path)]

def estimate_markov_model_from_sequences(sequences, num_states):
    # Estimate a Markov model based on the sequences (integers) provided.

    # pi[i] = Pr(s_0 = i)
    pi_counts = np.zeros(num_states)

    # A[i, j] = Pr(s_t = j | s_{t-1} = i)
    A_counts = np.zeros((num_states, num_states))
    
    for n, sequence in enumerate(sequences):
        prev_char = None
        for n_char, char in enumerate(sequence):
            if n_char == 0:
                pi_counts[char] += 1
            
            if prev_char is not None:
                A_counts[prev_char, char] += 1
            
            prev_char = char
#         assert False, "Collect counts for pi and A and return parameter estimates."
#     return pi_counts, A_counts
    pi = pi_counts / pi_counts.sum()
    A = A_counts / A_counts.sum(axis=0)
    assert(np.allclose(A.argmax(axis=0), A_counts.argmax(axis=0)))
    
    return pi, A

**TASK 1**: Make the following block run by completing the method 'estimate_markov_model_from_sequences' above.

In [8]:
# Some data to use.
plaintext = 'plaintext/english.txt'
# plaintext = 'plaintext/shakespeare.txt'
# plaintext = 'plaintext/russian.txt'

ciphertext = 'encrypted/1_encrypted.txt' # short sequences in english
# ciphertext = 'encrypted/99_encrypted.txt' # longer sequences in russian

# load a character to integer mapping and reverse                                                                                                         
char_to_int_mapping, int_to_char_mapping = get_char_to_int_mapping(plaintext)

# load sequences as ints                                                                                                                                  
plaintext_sequences = load_sequences(plaintext, char_to_int_mapping)
encrypted_sequences = load_sequences(ciphertext, char_to_int_mapping)

# estimate a markov model over characters                                                                                                                 
pi, A = estimate_markov_model_from_sequences(plaintext_sequences, len(char_to_int_mapping))

Below is a mostly implemented HMM.

In [9]:
class HMM():

    def __init__(self, observations_to_char_mapping={}, states_to_char_mapping={}):
        # Determine number of states and observation space. 
        self.num_states = len(states_to_char_mapping)
        self.num_outputs = len(observations_to_char_mapping)
        self.states_to_char_mapping = states_to_char_mapping
        self.observations_to_char_mapping = observations_to_char_mapping
       
        # Random initialization
        self.pi = np.random.rand(self.num_states)
        self.pi /= np.sum(self.pi)
        self.A = np.random.rand(self.num_states, self.num_states)
        self.A /= np.sum(self.A, 1, keepdims=True)
        self.B = np.random.rand(self.num_states, self.num_outputs)
        self.B /= np.sum(self.B, 1, keepdims=True) 
        
    def estimate_with_em(self, sequences, parameters={}, epsilon=0.001, max_iters=100):
        # Estimates all parameters not provided in 'parameters' based on 'sequences'.
        self.fixed_pi = 'pi' in parameters
        if self.fixed_pi:
            self.pi = parameters['pi']
        self.fixed_A = 'A' in parameters
        if self.fixed_A:
            self.A = parameters['A']
        self.fixed_B = 'B' in parameters
        if self.fixed_B:
            self.B = parameters['B']
    
        previous_llh = None
        iter = 0
        while True and iter < max_iters:
            # Infer expected counts.
            pi_counts, A_counts, B_counts, log_likelihood = self.e_step(sequences)

            # Update parameters based on counts.
            self.m_step(pi_counts, A_counts, B_counts)

            # Output some sequences for debugging.
            self.output(sequences[:10])

            # Log likelihood should be increasing
            print('iteration %d; log likelihood %.4f' % (iter, log_likelihood))
            if previous_llh:
                assert log_likelihood >= previous_llh
                if log_likelihood - previous_llh < epsilon:
                    break
            previous_llh = log_likelihood
        
            iter += 1


    def e_step(self, sequences):
        # Reset counters of statistics
        pi_counts = np.zeros_like(self.pi)
        A_counts = np.zeros_like(self.A) 
        B_counts = np.zeros_like(self.B) 
        total_log_likelihood = 0.0

        for sequence in sequences:
            # Run Forward-Backward dynamic program
            alpha, beta, gamma, xi, log_likelihood = self.forward_backward(sequence)
  
            # Accumulate statistics.
            pi_counts += gamma[:, 0]
            A_counts += xi
            for t, x in enumerate(sequence):
                B_counts[:, x] += gamma[:, t]
            
            total_log_likelihood += log_likelihood

        return pi_counts, A_counts, B_counts, total_log_likelihood

    def m_step(self, pi_counts, A_counts, B_counts):
        if not self.fixed_pi:
            self.pi = pi_counts / np.sum(pi_counts)
        if not self.fixed_A:
            self.A = A_counts / np.sum(A_counts, 1, keepdims=True)
        if not self.fixed_B:
            self.B = B_counts / np.sum(B_counts, 1, keepdims=True)
        
    def max_posterior_decode(self, sequence):
        _, _, gamma, _, log_likelihood = self.forward_backward(sequence)
        return np.argmax(gamma, 0)
        
    def forward_backward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = self.forward(sequence)
        
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = self.backward(sequence)

        # gamma[i][t] = p(z_t = i|x_1, ..., x_T)
        gamma = (alpha * beta) / np.sum(alpha * beta, 0)

        # xi[i][j] = p(z_t = i, z_{t+1} = j|x_1, ..., x_T)
        xi = np.zeros_like(self.A)
        for t in range(1, len(sequence)-1):
            this_xi = np.zeros_like(self.A)
            for i in range(self.num_states):
                for j in range(self.num_states):
                    this_xi[i, j] += alpha[i, t] * self.A[i, j] * beta[j, t+1] * self.B[j, sequence[t+1]]        
            xi += this_xi / np.sum(this_xi)
                
        return alpha, beta, gamma, xi, np.log(np.sum(alpha[:, len(sequence)-1]))

    def forward(self, sequence):
        # alpha[i][t] = p(x_1, ..., x_t, z_t = i)
        alpha = np.zeros((len(self.pi), len(sequence)))
        for t_plus1, x_t_plus1 in enumerate(sequence):
            for i in range(self.num_states):
                if t_plus1 == 0:
                    x_0 = x_t_plus1
                    
                    alpha[i, 0] = self.pi[i] * self.B[i, x_0]
                else:
                    new = 0.0
                    for j in range(self.num_states):
                        new += alpha[j, t_plus1 - 1] * self.A[j, i]
                        
                    alpha[i][t_plus1] = self.B[i, x_t_plus1] * new
                    
#         assert False, "Implement forward recursion"
        return alpha 
    
    def backward(self, sequence):
        # beta[i][t] = p(x_t+1, ..., x_T|z_t = i)
        beta = np.zeros((len(self.pi), len(sequence)))
        
        for ind, x_t_plus1 in enumerate(sequence[::-1]):
            t_plus1 = len(sequence) - ind
            
            for i in range(self.num_states):
                t = t_plus1 - 1
                if t_plus1 == len(sequence):
                    beta[i][t] = 1.0
                else:
                    new = 0.0
                    for j in range(self.num_states):
                        new += beta[j, t_plus1] * self.A[i, j] * self.B[j, x_t_plus1]
                        
                    beta[i][t] = new
#             print(beta[0, :])
        
#         assert False, "Implement backwards recursion to compute betas."
        return beta

    def output(self, sequences):
        # Output some decoded states. 
        for i, sequence in enumerate(sequences):
            observations = [self.observations_to_char_mapping[x] for x in sequence]                
            map_states = [self.states_to_char_mapping[x] for x in self.max_posterior_decode(sequence)]
            print('(states):       %s\n(observations): %s' % (''.join(map_states), ''.join(observations)))


**TASK 2**: Implement the assertions in 'forward' and 'backward' methods on the HMM class so that the following block passes.

In [10]:
# Since it's a substitution cipher we assume hidden states and observations have same alphabet.
state_to_char_mapping = int_to_char_mapping
observation_to_char_mapping = int_to_char_mapping

# Initialize a HMM with the correct state/output spaces.
hmm = HMM(observation_to_char_mapping, state_to_char_mapping)

# Estimate the parameters and decode the encrypted sequences.
hmm.estimate_with_em(encrypted_sequences[:10], parameters={'A' :A,'pi':pi})

(states):       th     le    e  z
(observations): noeixjtcoxexhwfei
(states):       a       an     a r   e   j
(observations): cejjgtjxkhtntjxkegnxiwnxfq
(states):       aee  e    o t  e  we t e h   we t  az
(observations): gddxiwnxqi thqxgnxbwtxpwhvqnxbwthjqdp
(states):       te h oe     ee e       eeoe t w
(observations): nwxoq vqxfqxgixgxefxexjwd gqhxg
(states):       teo n ee wn e  e   ae t  h   we t  az
(observations): wd qhxgixyhecngcqxekdqhxnoeixbwthjqdp
(states):       te     tee oe  e x
(observations): nwxfeaqxcwi gngwij
(states):       an   s he  e we t n   e  e   r q
(observations): khtntjxvwxnwxbwtxehqxiwnxcejjgtj
(states):       a       e  q
(observations): cejjgtjxgxef
(states):       an     e   i we t n  eex
(observations): khtntjxgxjebxbwtxehqxiwn
(states):       a        th t    e  e   e  h ee  e h    r  az
(observations): cejjgtjxthvqxfqxiwxfwhqxgxjoeddxpwhvqnxfbjqdp
iteration 0; log likelihood -909.3370
(states):       ah     me  t e  j
(observations): noeixjtcoxexhw

AssertionError: 

**TASK 3**: Some of the encrypted sequences are quite long. Try decoding some from 'encrypted/99_encrypted.txt' (note these are in Russian).

**TASK 4**: Make your implementation of forward and backward more efficient by removing all but the outermost for-loop.

**TASK 5**: Try to classify the author of each text. 